# KNN - Implementação do algoritmo K-vizinhos mais próximos

In [3]:
import pandas as pd

In [4]:
base = pd.read_csv("base_conhecimento.csv")
classificar = pd.read_csv("registros_para_classificar.csv")

In [5]:
def dist_euclidiana(a : list, b : list) -> float | None:
    if len(a) != len(b):
        return
    return (sum([(m - n)**2 for m, n in zip(a,b)])) * (1/2)

- Abaixo, a implementação do algoritmo KNN

In [58]:
def k_vizinhos_proximos(base, classificacao, k : int):
    base_tratada = base.drop(columns=["Species"]).values
    classificacao = classificacao.values
    
    classes = []
    for i in range(len(classificacao)):
        distancias = [dist_euclidiana(classificacao[i], a) for a in base_tratada]
        mais_proximo = distancias.index(min(distancias))
        distancias = pd.DataFrame(distancias)
        distancias.columns = ["Value"]
        k_vizinhos = distancias.sort_values(by="Value").tail(k)

        counter = {
                "Iris-setosa" : 0,
                "Iris-versicolor" : 0,
                "Iris-virginica" : 0
            }

        for index, j in k_vizinhos.iterrows():
            if base.iloc[index]["Species"] == "Iris-virginica":
                counter["Iris-virginica"] += 1
            elif base.iloc[index]["Species"] == "Iris-versicolor":
                counter["Iris-versicolor"] += 1
            elif base.iloc[index]["Species"] == "Iris-virginica":
                counter["Iris-setosa"] += 1
        
        maior_ocorrencia = max(counter.items(), key=lambda k:k[1])

        values = counter.values()
        if len(values) != len(set(values)):
            classes.append(base.iloc[mais_proximo]["Species"])
        else:
            classes.append(maior_ocorrencia)

    return pd.DataFrame(classes)

- Abaixo, uma função que verifica cada possibilidade de k com a resposta ideal em _respostas-em-ordem.csv_

In [80]:
def calcula_erros(base, classificacao):
    base_tratada = base.drop(columns=["Species"]).values

    erros = []
    respostas = pd.read_csv("respostas_em_ordem.csv")
    for n in range(1, len(base_tratada) + 1):
        classes = k_vizinhos_proximos(base, classificacao, n)

        erro = 0
        for m in range(len(classes)):
            if classes[0][m] != respostas["Species"][m]:
                erro += 1

        erros.append(erro)

    return erros.index(min(erros)) + 1      

In [81]:
print(calcula_erros(base, classificar))

1


- Abaixo, é tomado o valor com menos erros e cria um csv, mostrando suas respostas.

In [83]:
def main():
    menor_erro = calcula_erros(base, classificar)
    classificacao = k_vizinhos_proximos(base, classificar, menor_erro)
    classificacao = pd.DataFrame(classificacao)
    classificacao.columns = ["Species"]
    classificacao.to_csv(f"melhor_classificacao_k{menor_erro}.csv")

if __name__ == "__main__":
    main()